<a href="https://colab.research.google.com/github/kengmingcheng/Concrete_Strength_Prediction/blob/master/UCLA_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.utils import shuffle

In [ ]:
# Code to read csv file into Colaboratory:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = "https://drive.google.com/open?id=1niFDtDbD-NtI0fINwxGidgAoVgRI5r1g"
fluff, id_down = link.split('=')
print("File ID: {}".format(id_down))

upload_link = "https://drive.google.com/open?id=1OPkyRvIKKjor9RspmfJ2eP9c02rXNzKV"
url, id_up = link.split('=')
print("Upload folder ID: {}".format(id_up))

file = drive.CreateFile({'id':id_down}) 
file.GetContentFile('clean_data_28day.csv')

File ID: 1niFDtDbD-NtI0fINwxGidgAoVgRI5r1g
Upload folder ID: 1niFDtDbD-NtI0fINwxGidgAoVgRI5r1g


In [ ]:
df_data = pd.read_csv('clean_data_28day.csv')
df_data.describe()

,AEA Dose (oz/cwt),TYPE AWRA Dose (oz/cwt),W/(C+P),7 Day,28 Day,Coarse Agg Percentage,Fine Agg Percentage,%Fly Ash
count,7441.000000,7441.000000,7441.000000,7441.000000,7441.000000,7441.000000,7441.000000,7441.000000
mean,0.451765,3.298768,0.381364,4356.616046,5776.655019,47.554086,34.968076,3.010362
std,0.282533,1.884618,0.043183,977.603430,1032.851117,1.473127,2.249621,1.753022
min,0.000000,0.000000,0.240000,1720.000000,3200.000000,31.480118,25.965329,0.000000
25%,0.350000,2.010000,0.350000,3740.000000,5080.000000,47.001076,33.337954,1.738581
50%,0.460000,3.990000,0.380000,4200.000000,5640.000000,47.877294,34.767074,3.358313
75%,0.580000,4.920000,0.400000,4830.000000,6340.000000,48.413908,36.413717,3.703186
max,3.770000,6.080000,0.590000,35790.000000,12050.000000,54.070968,52.551303,6.511770


In [ ]:
df_data.columns

Index(['AEA Dose (oz/cwt)', 'TYPE AWRA Dose (oz/cwt)', 'W/(C+P)', '7 Day',
       '28 Day', 'Coarse Agg Percentage', 'Fine Agg Percentage', '%Fly Ash'],
      dtype='object')

In [ ]:
input_features=['AEA Dose (oz/cwt)', 'TYPE AWRA Dose (oz/cwt)', 'W/(C+P)',
               'Coarse Agg Percentage', 'Fine Agg Percentage', '%Fly Ash'
                ]
df_data = shuffle(df_data, random_state = 42)

y7_data=df_data[["7 Day"]]
y28_data=df_data[["28 Day"]]
# Select scaler for input and target data
input_scaler = MinMaxScaler()
target_scaler = StandardScaler()
# scaler transformation
# input_data=input_scaler.fit_transform(df_data.loc[:,input_features])
input_data=input_scaler.fit_transform(df_data.drop(["7 Day", "28 Day"], axis=1))
target_data=target_scaler.fit_transform(y28_data)
# split data into training and test sets
train_index, cv_index = train_test_split(np.arange(len(df_data)), random_state= 42, test_size=0.2)

train_input=input_data[train_index]
cv_input=input_data[cv_index]

train_target=target_data[train_index]
cv_target=target_data[cv_index]

In [ ]:
nn_sklearn = MLPRegressor(hidden_layer_sizes=(11),activation='logistic',max_iter=1000,random_state=1,solver='lbfgs')

In [ ]:
nn_sklearn.fit(train_input, train_target)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=11, learning_rate='constant',
             learning_rate_init=0.001, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [ ]:
nn_sklearn.predict(cv_input)

array([-0.23317678, -0.2718886 , -0.14095829, ..., -0.33236417,
       -0.35085145, -0.34491842])

In [ ]:
Kfold = KFold(n_splits=5, random_state = 42, shuffle = True)
r_squared = cross_val_score(nn_sklearn, input_data, target_data, cv = Kfold, scoring='r2')
print(np.average(r_squared))

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d

0.5971561740572776


In [ ]:
scaler = StandardScaler()
raw_data = pd.read_csv("clean_data_28day.csv")
raw_data = shuffle(raw_data, random_state = 42)
X = raw_data.drop(["7 Day", "28 Day"], axis = 1)
XX = MinMaxScaler().fit_transform(X)
scaler.fit(raw_data)
raw_data = scaler.transform(raw_data)
y28 = raw_data[:,4]
y7 = raw_data[:,3]
X7_train, X7_test, y7_train, y7_test = train_test_split(X, y7, test_size=0.2, random_state=42)
X28_train, X28_test, y28_train, y28_test = train_test_split(X, y28, test_size=0.2, random_state=42)
est = MLPRegressor(hidden_layer_sizes=(11),activation='logistic',max_iter=1000,random_state=1,solver='lbfgs')
r2 = cross_val_score(est, XX, y28, cv = 5)
print(np.average(r2))

0.5954855708547134


In [ ]:
mse = cross_val_score(est,XX, y28, cv=5, scoring = 'neg_mean_squared_error')
print(np.average(np.sqrt(np.negative(mse))))

0.6355120332873878


In [ ]:
test = df_data[["28 Day","7 Day"]]
test_data = StandardScaler().fit_transform(test)